In [57]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

In [64]:
def extractHTML(HTMLcards, username, tweets_list, max_tweets):
    if '@' not in username:
        username = '@' + username
        
    for card in HTMLcards:
        if len(tweets_list) == max_tweets:
            break
        handle = card.find_element(By.XPATH, './/span[contains(text(), "@")]').text
        tweet = card.find_element(By.XPATH, './/div[2]/div[2]/div[2]').text
        if handle != username:
            continue
        if tweet in tweets_list:
            continue
        tweets_list.append(tweet)

    return tweets_list

In [65]:
def getTweets(username, max_tweets=50):

    # Configure Chrome options for headless browsing
    chrome_options = Options()
    chrome_options.add_argument("--headless")  # Run Chrome in headless mode
    chrome_options.add_argument("--disable-gpu")  # Disable GPU acceleration

    # Create a WebDriver instance with headless Chrome
    driver = webdriver.Chrome(options=chrome_options)

    driver.get(f"https://twitter.com/{username}")

    time.sleep(2)

    tweets_list = []
    retry_counter = 0
    while len(tweets_list) < max_tweets:
        if retry_counter >= 3:
            break
        
        window_height = driver.execute_script("return window.innerHeight;")
        num_tweets = len(tweets_list)
        # print (num_tweets)

        HTMLcards = driver.find_elements(By.XPATH, '//article[@data-testid="tweet"]')

        tweets = extractHTML(HTMLcards, username, tweets_list=tweets_list, max_tweets=max_tweets)
        tweets_list = tweets

        if num_tweets == len(tweets_list):
            retry_counter += 1
            # print (f'retry: {retry_counter}')
        else:
            retry_counter = 0
            
        driver.execute_script(f"window.scrollBy(0, {5 * window_height});")

        time.sleep(1.25)

    driver.quit()

    return tweets_list

In [68]:
tweets = getTweets('shalin_1', max_tweets=50)